# Pipeline

In [1]:
###imports
import pandas as pd
import numpy as np

In [2]:
###load the dataset
def get_data(path):
    df = pd.read_csv(path)
    y = df['classification']
    X = df.drop(columns='classification')
    
    return X,y
X,y = get_data("../raw_data/kidney_disease.csv")


In [3]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:100,:2]
X.shape


(100, 2)

In [4]:
y = iris.target[:100]
y.size

100

In [5]:
def set_pipeline('''self'''):
    """defines the pipeline as a class attribute"""

    dist_pipe = Pipeline([
        ('dist_trans', DistanceTransformer()),
        ('stdscaler', StandardScaler())
    ])
    time_pipe = Pipeline([
        ('time_enc', TimeFeaturesEncoder('pickup_datetime')),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ])
    preproc_pipe = ColumnTransformer([
        ('distance', dist_pipe, ["pickup_latitude", "pickup_longitude", 'dropoff_latitude', 'dropoff_longitude']),
        ('time', time_pipe, ['pickup_datetime'])
    ], remainder="drop")
    self.pipe = Pipeline([
        ('preproc', preproc_pipe),
        ('model', model)
    ])
    return self.pipe

SyntaxError: invalid syntax (3706515062.py, line 1)

# Models
    

## Log Regression

In [ ]:

#------------paramenters of logreg----------------#
#C: float, default=1.0, Inverse of regularization strength (smaller: higher regularization)

###dualbool: default=False
###set only to true for small df, where n_features > n_samples:

###l1_ratio:
#The Elastic-Net mixing parameter, with 0 <= l1_ratio <= 1

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold

In [18]:
model = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# define search space
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]

scoring = ['accuracy', 'f1']

# define search
search = GridSearchCV(model, space, scoring=scoring, n_jobs=-1, cv=cv)
result = search.fit(X,y)
result.cv_results_
result.best_estimator_

ValueError: For multi-metric scoring, the parameter refit must be set to a scorer key or a callable to refit an estimator with the best parameter setting on the whole data and make the best_* attributes available for that metric. If this is not needed, refit should be set to False explicitly. True was passed.

/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to co

In [ ]:


#def grid_search_log_reg(X, y):
'''grid-searches a log model and returns a 
model with the best params and the cv_results'''

# instantiate model
model = LogisticRegression()

pipe = Pipeline([('classifier' , LogisticRegression())])

#instanziate grid search
param_grid = [{'classifier': [LogisticRegression()],
               'classifier_solver': ['newton-cg', 'lbfgs', 'sag'], 
               'classifier_penalties': ['l2', 'none']},
              {'classifier': [LogisticRegression()],
              'classifier_solver': ['liblinear'],
              'classifier_penalties': ['l1', 'l2']},
              {'classifier': [LogisticRegression()],
              'classifier_solver': ['saga'],
              'classifier_penalties': ['elasticnet', 'l1', 'l2', 'none']}
             ]

search = GridSearchCV(pipe, param_grid, scoring='r2', return_train_score=True, n_jobs=-1)
result = search.fit(X,y)
    
    #return [search.best_estimator_, search.cv_results_]

#grid_search_log_reg(X, y)

result


In [8]:
grid = { 'l1_ratio': [0.2, 0.5, 0.8]}

# Instanciate Grid Search
search = GridSearchCV(model, grid, 
                           scoring = 'r2',
                           cv = 5,
                           n_jobs=-1 # paralellize computation
                          )
result = search.fit(X,y)
result

ERROR! Session/line number was not unique in database. History logging moved to new session 308


/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'l1_ratio': [0.2, 0.5, 0.8]}, scoring='r2')

In [9]:

serach = GridSearchCV(model, param_grid, return_train_score=True, n_jobs=-1)
result = search.fit(X,y)

/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


In [10]:
result

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'l1_ratio': [0.2, 0.5, 0.8]}, scoring='r2')

In [11]:
###grid_search
penalties = ['none','l2','l1','elasticnet']
solvers = {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}

###allowed penalties by solver:
'''
‘newton-cg’ - [‘l2’, ‘none’]

‘lbfgs’ - [‘l2’, ‘none’]

‘liblinear’ - [‘l1’, ‘l2’]

‘sag’ - [‘l2’, ‘none’]

‘saga’ - [‘elasticnet’, ‘l1’, ‘l2’, ‘none’]
'''

#max_iterint, default=100
#Maximum number of iterations taken for the solvers to converge.

'\n‘newton-cg’ - [‘l2’, ‘none’]\n\n‘lbfgs’ - [‘l2’, ‘none’]\n\n‘liblinear’ - [‘l1’, ‘l2’]\n\n‘sag’ - [‘l2’, ‘none’]\n\n‘saga’ - [‘elasticnet’, ‘l1’, ‘l2’, ‘none’]\n'

In [12]:
###confusion_matrix:
results_df = pd.DataFrame({"actual": y_test,
                           "predicted": preds})#Store results in dataframe

confusion_matrix = pd.crosstab(index= results_df['actual'],
                               columns = results_df['predicted'])

NameError: name 'y_test' is not defined

In [ ]:
param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
]

/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/viktoria/.pyenv/versions/3.8.12/envs/kidney_kids/lib/python3.8/sit